In [1]:
import numpy as np
import numpy.random as npr
from sklearn.datasets import load_boston
from sklearn.ensemble import AdaBoostClassifier as adaBoost
from sklearn.tree import DecisionTreeClassifier

# AdaBoost:

The idea of AdaBoost is to combine $M$ weak learners $g_1,\dots,g_M$ into a strong one, in a binary classification setup. By weak learners we mean classifiers that perform only slightly better than random guessing, and by strong one, one that performs significantly better. A requirement for using AdaBoost is that we should be able to train the weak learners with a weighted data set, that is, each point in the loss can have a different weight $\omega_i$. Furthermore, it will only be useful when the weak learners are significantly different from each other. The algorithm is as follows:

1. Initialize the weights with $\omega_i=1/n$ for $i=1,\dots,n$ (where $n$ is the number of data points).

2. For m=1 to M:

 2.1 Fit a classifier $g_m$ to the training data using weights $\omega_i$ (for example, by minimizing $err_m$ as defined below).
 
 2.2 Compute the weighted misclassification error: $err_m = \dfrac{\sum_{i=1}^n \omega_i \mathbb{1}(y_i \neq g_m(x_i))}{\sum_{i=1}^n \omega_i}$.

 2.3 Compute $\alpha_m = \log\Big(\dfrac{1-err_m}{err_m}\Big)$ (decreasing transformation of $err_m$).
 
 2.4 Update the weights $\omega_i \leftarrow \omega_i \exp(\alpha_m \mathbb{1}(y_i \neq g_m(x_i)))$ for $i=1,2,\dots, n$. Note that if the point is classified correctly this does not update the weight, otherwise the weight is multiplied by $\exp(\alpha_m)$. When the weight is updated (i.e. misclassified), it is increased if and only if $err_m < 0.5$. We should expect $err_m < 0.5$ (since otherwise the classifier would be so bad that we could get a better classifier by simply predicting the opposite of what the classifier predicts), so updating should increase the weight, thus encouraging the next weak learner to classify that point correctly.
 
3. Output $f(x) = sign\Big(\sum_{m=1}^M \alpha_m g_m(x)\Big)$, which is weighting learners with small error higher than those with large errors.

Note that it is indeed important for the weak learners to be significantly different: if they are all very similar, then their weighted average $\sum_{m=1}^M \alpha_m g_m(x)$ will be similar to each individual term, thus not resulting in a strong learner since the classifier is going to be similar to every weak learner. This makes trees a very natural fit for the weak learners: if you recall from the tree-based methods tutorial, we mentioned that small changes in the data could result in significantly different trees, so we should expect that trees trained with different weights to be different as well.

Now, we will go through an example using the Boston Housing dataset. The data consists of 506 suburbs of Boston. At each suburb, 13 variables were measured, in addition to median value of occupied households (in \$1000s). To make the variable binary, we assign $-1$ when the value is <30 and $1$ otherwise. We also split the data into train and test parts:

In [2]:
X, y = load_boston(return_X_y = True)
y = 2 * (y >= 30) - 1
n = np.shape(X)[0] #506
n_train = 400

npr.seed(2480)
train = np.zeros(n, dtype=np.bool)
n_p2 = np.sum(y == 1)
train[npr.choice(n, n_train, replace=False)] = True
test = np.logical_not(train)
n_test = np.sum(test)
n_train_p = np.sum(y[train] == 1)
n_train_n = np.sum(y[train] == -1)
n_test_p = np.sum(y[test] == 1)
n_test_n = np.sum(y[test] == -1)

k = 5
train_idx = np.nonzero(train)[0]
npr.shuffle(train_idx)
cv_grid = np.linspace(0, n_train, k + 1).astype(np.int32)

Then, we train a decision tree using $k$-fold cross valitation to select the max_length

In [3]:
depth_params_length = 10
depth_params = np.arange(1, depth_params_length+1)
cv_err_tree = np.empty(depth_params_length)
train_cv = np.copy(train)
for i in xrange(depth_params_length):
    print "tuning parameter: ", i
    tree = DecisionTreeClassifier(max_depth=depth_params[i])
    tmp_err = 0.0
    for j in xrange(k):
        train_cv[train_idx[cv_grid[j] : cv_grid[j + 1]]] = False
        tree.fit(X[train_cv, :], y[train_cv])
        y_pred_k = tree.predict(X[train_idx[cv_grid[j] : cv_grid[j + 1]]])
        tmp_err += np.sum(y_pred_k != y[train_idx[cv_grid[j] : cv_grid[j + 1]]])
        train_cv[train_idx[cv_grid[j] : cv_grid[j + 1]]] = True
    cv_err_tree[i] = tmp_err / k

depth_param_opt_tree = depth_params[np.argmin(cv_err_tree)]
print "CV selected depth: ", depth_param_opt_tree

tuning parameter:  0
tuning parameter:  1
tuning parameter:  2
tuning parameter:  3
tuning parameter:  4
tuning parameter:  5
tuning parameter:  6
tuning parameter:  7
tuning parameter:  8
tuning parameter:  9
CV selected depth:  8


Train the selected tree from CV and look at the test performance:

In [4]:
tree = DecisionTreeClassifier(max_depth=depth_param_opt_tree)
tree.fit(X[train, :], y[train])
y_test_pred_tree = tree.predict(X[test, :])
print "False positive", np.sum(np.logical_and(y[test] == -1, y_test_pred_tree == 1)) / float(n_test_n)
print "True positive", np.sum(np.logical_and(y[test] == 1, y_test_pred_tree == 1)) / float(n_test_p)
print "False negative", np.sum(np.logical_and(y[test] == 1, y_test_pred_tree == -1)) / float(n_test_p)
print "True negative", np.sum(np.logical_and(y[test] == -1, y_test_pred_tree == -1)) / float(n_test_n)

False positive 0.0340909090909
True positive 0.833333333333
False negative 0.166666666667
True negative 0.965909090909


Now, we use AdaBoost with decision trees of depth 3 as weak learners. We use CV to select the number of weak learners. Note that we could also use cross validation to select the depth parameter of the decision trees instead of setting it to 3.

In [5]:
M_params_length = 10
M_params = np.arange(100, (M_params_length)*10 + 100, 10)
cv_err_adaB = np.empty(M_params_length)
train_cv = np.copy(train)
for i in xrange(M_params_length):
    print "tuning parameter: ", i
    adaB = adaBoost(DecisionTreeClassifier(max_depth=3), n_estimators=M_params[i])
    tmp_err = 0.0
    for j in xrange(k):
        train_cv[train_idx[cv_grid[j] : cv_grid[j + 1]]] = False
        adaB.fit(X[train_cv, :], y[train_cv])
        y_pred_k = adaB.predict(X[train_idx[cv_grid[j] : cv_grid[j + 1]]])
        tmp_err += np.sum(y_pred_k != y[train_idx[cv_grid[j] : cv_grid[j + 1]]])
        train_cv[train_idx[cv_grid[j] : cv_grid[j + 1]]] = True
    cv_err_adaB[i] = tmp_err / k

M_opt_adaB = M_params[np.argmin(cv_err_adaB)]
print "CV selected M: ", M_opt_adaB

tuning parameter:  0
tuning parameter:  1
tuning parameter:  2
tuning parameter:  3
tuning parameter:  4
tuning parameter:  5
tuning parameter:  6
tuning parameter:  7
tuning parameter:  8
tuning parameter:  9
CV selected M:  130


In [6]:
adaB = adaBoost(DecisionTreeClassifier(max_depth=3), n_estimators=M_opt_adaB)
adaB.fit(X[train, :], y[train])
y_test_pred_adaB = adaB.predict(X[test, :])
print "False positive", np.sum(np.logical_and(y[test] == -1, y_test_pred_adaB == 1)) / float(n_test_n)
print "True positive", np.sum(np.logical_and(y[test] == 1, y_test_pred_adaB == 1)) / float(n_test_p)
print "False negative", np.sum(np.logical_and(y[test] == 1, y_test_pred_adaB == -1)) / float(n_test_p)
print "True negative", np.sum(np.logical_and(y[test] == -1, y_test_pred_adaB == -1)) / float(n_test_n)

False positive 0.0340909090909
True positive 0.888888888889
False negative 0.111111111111
True negative 0.965909090909
